In [10]:
import numpy as np
import pandas as pd
import xarray as xr
from mpl_toolkits.basemap import Basemap
import glob
import os
import shutil
import re
import cartopy
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
#
%matplotlib inline
# %matplotlib notebook
#
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap, BoundaryNorm
#
# PD=os.getcwd(); PD


### 
### ==>
### <font color=black> Function:: <font color=blue> **Difference** </font> </font>

In [11]:
for i, wf in enumerate(WF):
    fileG='total-workflow.nc'
    fileP='nzOG.{arg0}.total.workflow.wf{arg1}.local.nc'.format(arg0=ssp,arg1=wf.split('_')[1])
    var='sea_level_change'    
    #
    folder='dist_workflows'
    pathG='/projects/kopp/ggg46/nz_projections/regional/{arg0}/{arg1}/{arg2}/'.format(arg0=folder,arg1=wf,arg2=ssp)
    pathP='/scratch/pk695/FACTS/nzOG-pk695/local-pk695/002_{arg0}/{arg1}/{arg2}/'.format(arg0=folder,arg1=wf,arg2=ssp) 

!ls $pathP
! echo "xXxXxX"
!ls $pathG

file_list = os.listdir(pathP)
file_list[1]


nzOG.ssp126.ar5glaciers.ipccar5.glaciers_localsl.nc
nzOG.ssp126.bamber19.bamber19.icesheets_AIS_localsl.nc
nzOG.ssp126.bamber19.bamber19.icesheets_GIS_localsl.nc
nzOG.ssp126.lws.ssp.landwaterstorage_localsl.nc
nzOG.ssp126.NZInsarGPS.NZInsarGPS.verticallandmotion_localsl.nc
nzOG.ssp126.ocean.tlm.sterodynamics_localsl.nc
nzOG.ssp126.total.workflow.wf4.local.nc
xXxXxX
glaciers-ar5-glaciersgmip2-ssp126_localsl.nc
icesheets-ipccar6-bambericesheet-ssp126_AIS_localsl.nc
icesheets-ipccar6-bambericesheet-ssp126_GIS_localsl.nc
landwaterstorage-ssp-landwaterstorage-ssp126_localsl.nc
oceandynamics-tlm-oceandynamics-ssp126_localsl.nc
total-workflow.nc
verticallandmotion-NZ_InsarGPS-verticallandmotion_localsl.nc


'nzOG.ssp126.NZInsarGPS.NZInsarGPS.verticallandmotion_localsl.nc'

In [15]:
import glob

folder_path = pathG
search_term = "ocean"   # "glaciers"  # replace with the word you want to search for
file_pattern = f"{folder_path}/*{search_term}*"  # create a pattern to match files containing the search term
matching_files = glob.glob(file_pattern)
if len(matching_files)>1: 
    raise ValueError("There are 2 files with same keyword")
fnme = os.path.basename(matching_files[0])
print(fnme)    
    

oceandynamics-tlm-oceandynamics-ssp126_localsl.nc


In [3]:
# matching_files

ssp='ssp126'
if ssp=='ssp126' or ssp=='ssp585':
    WF=['wf_1e','wf_1f','wf_2e','wf_2f','wf_3e','wf_3f','wf_4']
elif ssp=='ssp119' or ssp=='ssp245':    
    WF=['wf_1e','wf_1f','wf_2e','wf_2f','wf_3e','wf_3f']
elif ssp=='ssp370':
    WF=['wf_1e','wf_1f','wf_2e','wf_2f']

In [ ]:
#Choice var

DataAIS
glaciers
GrIS || GIS
landwaterstorage
verticallandmotion
ocean
total




ssp='ssp126'
#
stnLEN=7434
#
ptile=[17,50,83]
#
var='sea_level_change'


def diffDATA(ssp,stnLEN,ptile):

    if ssp==ssp126 or ssp==ssp585:
        WF=['wf_1e','wf_1f','wf_2e','wf_2f','wf_3e','wf_3f','wf_4']
    elif ssp==ssp119 or ssp==ssp245:    
        WF=['wf_1e','wf_1f','wf_2e','wf_2f','wf_3e','wf_3f']
    elif ssp==ssp370:
        WF=['wf_1e','wf_1f','wf_2e','wf_2f']

    
    fig, axs = plt.subplots(nrows=WF.shape, ncols=ptile.shape, figsize=(12*2, 5*6))

    
    for i, wf in enumerate(WF):
        folder='dist_workflows'
        pathG='/projects/kopp/ggg46/nz_projections/regional/{arg0}/{arg1}/{arg2}/'.format(arg0=folder,arg1=wf,arg2=ssp)
        pathP='/scratch/pk695/FACTS/nzOG-pk695/local-pk695/002_{arg0}/{arg1}/{arg2}/'.format(arg0=folder,arg1=wf,arg2=ssp)
        #
        fileG='total-workflow.nc'
        fileP='nzOG.{arg0}.total.workflow.wf{arg1}.local.nc'.format(arg0=ssp,arg1=wf.split('_')[1])
    
    








        
    

    # labels
    EXP_G=pathG.split('/')[3].split('_')[0]; 
    EXP_P=pathP.split('/')[4].split('-')[1]; 
    EXP_file=fileP;  

    d_ncG, d_ncP = [xr.open_dataset(os.path.join(path, file)) for path, file in [(pathG, fileG), (pathP, fileP)]]

    yrST=2020; yrEN=2100; yr=np.arange(yrST, yrEN+1, 10)
    yrIDX_G, yrIDX_P=[np.searchsorted(d['years'].values, yr) for d in [d_ncG, d_ncP]]

    
    ptileIDX_G, ptileIDX_P=[np.searchsorted(d['quantiles'].values*100, ptile) for d in [d_ncG, d_ncP]]

    loc=np.arange(0,stnLEN) #Location:: choose from 0 to 7434         
    latP, lonP = [d_ncP[d][loc].values for d in ['lat', 'lon']]

    slcG=d_ncG[var][ptileIDX_G, yrIDX_G, loc]
    slcP=d_ncP[var][ptileIDX_P, yrIDX_P, loc]
    slcG_reshaped = slcG.values.reshape(-1)
    slcP_reshaped = slcP.values.reshape(-1)
    diff = slcG_reshaped - slcP_reshaped
    diff = np.reshape(diff, slcG.shape)

    xx=d_ncG['years'][yrIDX_G].values

    for pt0,pt1 in enumerate(ptile):
        row, col = i, pt0
        ax = axs[row, col]
        
        for lo0,lo1 in enumerate(loc):
            yy1=np.squeeze(diff[pt0,:,lo0])
            ax.plot(xx, yy1, label='')
            ax.grid(alpha=0.95)
            ax.text(.5, .9,ssp+' :: '+wf, fontsize=15, color='blue', transform=ax.transAxes, ha='left', va='center')
            ax.set_xlim(2020, 2100); ax.set_xticks(range(2020, 2100+1, 10))
            ax.tick_params(axis='y', labelsize=17)
            
            # Xlabels
            if row==6 and col in (0, 1, 2):
                ax.tick_params(axis='x', labelrotation=45,labelsize=17)
            
            # Ylabels
            if col==0 and row in (0, 1, 2, 3, 4, 5, 6):
                ax.set_ylabel('diff (mm)', fontsize=17); #ax.set_xlabel('Years')
            
            if row==0 and col in (0, 1, 2):
                QUANT=str(pt1)
                ax.set_title(QUANT+' Quantile Difference plot' ,fontsize=18)

plt.subplots_adjust(wspace=0.2, hspace=0.3)
plt.show()

### 
### ==>
### <font color=black> PLOT:: <font color=blue> **SSP126 all wf** </font> </font>

In [ ]:
# wf='wf_1e'
WF=['wf_1e','wf_1f','wf_2e','wf_2f','wf_3e','wf_3f','wf_4']
ssp='ssp126'

fig, axs = plt.subplots(nrows=7, ncols=3, figsize=(12*2, 5*6))

for i, wf in enumerate(WF):
    folder='dist_workflows'
    pathG='/projects/kopp/ggg46/nz_projections/regional/{arg0}/{arg1}/{arg2}/'.format(arg0=folder,arg1=wf,arg2=ssp)
    pathP='/scratch/pk695/FACTS/nzOG-pk695/local-pk695/002_{arg0}/{arg1}/{arg2}/'.format(arg0=folder,arg1=wf,arg2=ssp)

    fileG='total-workflow.nc'
    fileP='nzOG.{arg0}.total.workflow.wf{arg1}.local.nc'.format(arg0=ssp,arg1=wf.split('_')[1])
    var='sea_level_change'

    # labels
    EXP_G=pathG.split('/')[3].split('_')[0]; 
    EXP_P=pathP.split('/')[4].split('-')[1]; 
    EXP_file=fileP; 

    d_ncG, d_ncP = [xr.open_dataset(os.path.join(path, file)) for path, file in [(pathG, fileG), (pathP, fileP)]]

    yrST=2020; yrEN=2100; yr=np.arange(yrST, yrEN+1, 10)
    yrIDX_G, yrIDX_P=[np.searchsorted(d['years'].values, yr) for d in [d_ncG, d_ncP]]

    ptile=[17,50,83]
    ptileIDX_G, ptileIDX_P=[np.searchsorted(d['quantiles'].values*100, ptile) for d in [d_ncG, d_ncP]]

    loc=np.arange(0,7434) #Location:: choose from 0 to 7434
#     loc=np.arange(0,7) #Location:: choose from 0 to 7434           
    latP, lonP = [d_ncP[d][loc].values for d in ['lat', 'lon']]

    slcG=d_ncG[var][ptileIDX_G, yrIDX_G, loc]
    slcP=d_ncP[var][ptileIDX_P, yrIDX_P, loc]
    slcG_reshaped = slcG.values.reshape(-1)
    slcP_reshaped = slcP.values.reshape(-1)
    diff = slcG_reshaped - slcP_reshaped
    diff = np.reshape(diff, slcG.shape)

    xx=d_ncG['years'][yrIDX_G].values

    for pt0,pt1 in enumerate(ptile):
        row, col = i, pt0
        ax = axs[row, col]
        
        for lo0,lo1 in enumerate(loc):
            yy1=np.squeeze(diff[pt0,:,lo0])
            ax.plot(xx, yy1, label='')
            ax.grid(alpha=0.95)
            ax.text(.5, .9,ssp+' :: '+wf, fontsize=15, color='blue', transform=ax.transAxes, ha='left', va='center')
            ax.set_xlim(2020, 2100); ax.set_xticks(range(2020, 2100+1, 10))
            ax.tick_params(axis='y', labelsize=17)
            
            # Xlabels
            if row==6 and col in (0, 1, 2):
                ax.tick_params(axis='x', labelrotation=45,labelsize=17)
            
            # Ylabels
            if col==0 and row in (0, 1, 2, 3, 4, 5, 6):
                ax.set_ylabel('diff (mm)', fontsize=17); #ax.set_xlabel('Years')
            
            if row==0 and col in (0, 1, 2):
                QUANT=str(pt1)
                ax.set_title(QUANT+' Quantile Difference plot' ,fontsize=18)

plt.subplots_adjust(wspace=0.2, hspace=0.3)
plt.show()